
## Scenario
We decided to approach the activity by focusing on sentiment analysis but instead of 'positive' or 'negative' tweets we focused on determining whether a Slack message is 'Urgent', 'Not urgent' or 'Ambiguous'.

In our scenario we imagined this tool would be used by someone in a large company that has many people reporting to them via Slack on a regular basis. This user wants to make sure Slack is not a hinderance/distraction to their workday and want to be efficient with their time. 

The model will analyze the sentiment of the messages received in Slack and grade whether the user should address them soon ('Urgent') or if it's alright to address the message later ('Not urgent') or if it falls in a middle ground ('ambiguous').

In [1]:
import openai

api_key = open('key.txt','r').read().strip('\n')



# load and set our key
openai.api_key = api_key

In [2]:
message_history=[{"role": "system", "content": "You are a helpful assistant."}]


def gpt_response(inp, message_history, **params):
    # We save the user's input
    message_history.append({"role": "user", "content": f"{inp}"})
    # Generate a response from the chatbot model
    completion = openai.ChatCompletion.create(
      model = 'gpt-3.5-turbo',
      messages=message_history, 
      **params
    )
    # We save the assistant response
    message_history.append({"role": "assistant", "content": f"{completion.choices[0].message.content}"})
    return message_history

In [ ]:
#changed temp to 1 but overall still did well. We did a good job of giving the system clear instructions from the beginning.

message_history=[{"role": "system", "content": "You are a marketing expert for a fortune 500 company." 
                  "Decide whether a Slack message is urgent, not urgent or ambiguous based on it's relation to your ability to do your marketing job."},
             {"role": "user", "content": "Slack: \"I need you to create a presentation asap!\"\nSentiment:"},
              {"role": "assistant", "content": "Urgent"}
  ]
print("Write a Slack message: ")

while(True):
    message_history = gpt_response(input("> "),
                                   message_history,
                                   temperature=1,
                                   max_tokens=60,
                                   frequency_penalty=0.5)
    print(message_history[-1]["content"])

In [8]:
#alternate with no frequency penalty and removing "based on it's relation to your ability to do your marketing job"
# responses are more generic and more black and white, less likely to mark the 'ambiguous' messages.
message_history=[{"role": "system", "content": "You are a marketing expert for a fortune 500 company. Decide whether a Slack message is urgent, not urgent or ambiguous."},
             {"role": "user", "content": "Slack message: \"I need you to create a presentation asap!\"\nSentiment:"},
              {"role": "assistant", "content": "Urgent"}
  ]
print("Write a Slack message: ")

while(True):
    message_history = gpt_response(input("> "),
                                   message_history,
                                   temperature=0,
                                   max_tokens=60,
                                   frequency_penalty=0.0)
    print(message_history[-1]["content"])

Write a Slack message: 
> "Urgent: The ad copy has been rejected, please revise ASAP."
Urgent
> "The ad copy has been rejected, please revise ASAP."
Urgent
> There's a team lunch next Wednesday. RSVP if you haven't already.
Not urgent
> Does anyone have a charger I can borrow?
Not urgent
> Question: a trade organization wants to sponsor some brands inside our program, and I need help figuring out the billing of it. Do we just create a 2 part payment plan and charge the sponsor for the enrollment prices regardless of the brand’s participation? What am I missing here?
Ambiguous


KeyboardInterrupt: Interrupted by user

In [9]:
#this is our best model, we have specific instructions for the system and the parameters set.

message_history=[{"role": "system", "content": "You are a marketing expert for a fortune 500 company. Decide whether a Slack message is urgent, not urgent or ambiguousbased on it's relation to your ability to do your marketing job."},
             {"role": "user", "content": "Slack message: \"I need you to create a presentation asap!\"\nSentiment:"},
              {"role": "assistant", "content": "Urgent"}
  ]
print("Write a Slack message: ")

while(True):
    message_history = gpt_response(input("> "),
                                   message_history,
                                   temperature=0, #Controls the randomness of the output.
                                   max_tokens=12, #Limits the maximum number of tokens in the output.
                                   frequency_penalty=0.5) #Penalizes new tokens based on their existing frequency in the text so far.
    print(message_history[-1]["content"])

Write a Slack message: 
> "Urgent: The ad copy has been rejected, please revise ASAP."
Urgent
> "The ad copy has been rejected, please revise ASAP."
Urgent
> There's a team lunch next Wednesday. RSVP if you haven't already.
Not urgent
> Does anyone have a charger I can borrow?
Not urgent
> Question: a trade organization wants to sponsor some brands inside our program, and I need help figuring out the billing of it. Do we just create a 2 part payment plan and charge the sponsor for the enrollment prices regardless of the brand’s participation? What am I missing here?
Ambiguous


KeyboardInterrupt: Interrupted by user


## Findings and Lessons learned
- The more specific we were with our prompt for the system, the better and more consistent results we obtained.
- Since we were focusing on sentiment analysis,temperature was the most important parameter since we did not want any random results. max_tokens and frequency_penalty were less significant than if we were looking for a generative response
- we wanted to try using text-davinci-003 but were unable to because it has been deprecated.